In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from cohlib.jax.utils import jax_boilerplate
jax_boilerplate()

Platform:  cpu
28


In [3]:
import jax.numpy as jnp
import matplotlib.pyplot as plt

from cohlib.utils import pickle_open
from cohlib.confs.utils import omega
from cohlib.confs.latent import create_lrccn_basic_rank1

In [4]:
# L = 10 
# ov2 = -1.0
K = 3
L = 50
seed = 1
mu = 0.0
init = 'flat'
res_path = f'/projectnb/stephenlab/jtauber/cohlib/experiments/data/latent-basic_single_log/window1000/K{K}/L{L}/lseed{seed}/obs-pp_log/alpha{mu}/oseed{seed}/model-simple_inherit_latent_eigh/inherit-True/m_step-low-rank-eigh/{init}-init_eigvals-oracle_eigvecs-fit/newton-10_em-20/res.pkl'
obs_path = f'/projectnb/stephenlab/jtauber/cohlib/experiments/data/latent-basic_single_log/window1000/K{K}/L{L}/lseed{seed}/obs-pp_log/alpha{mu}/oseed{seed}/obs_sim.pkl'
latent_path = f'/projectnb/stephenlab/jtauber/cohlib/experiments/data/latent-basic_single_log/window1000/K{K}/L{L}/lseed{seed}/latent_sim.pkl'

# latent_load = pickle_open(latent_path)
# obs_load = pickle_open(obs_path)
res_load = pickle_open(res_path)
obs_load = pickle_open(obs_path)
latent_load = pickle_open(latent_path)


In [5]:
import cohlib.confs.utils as conf
from cohlib.jax.lr_model import LowRankToyModel
obs = obs_load['obs']
zs_nz = latent_load['zs_nz']
cfg = omega(res_load['cfg'])
lcfg = cfg.latent
mcfg = cfg.model
ocfg = cfg.obs

N = lcfg.num_freqs
freqs = jnp.arange(N)
nz_model = jnp.array([lcfg.target_freq_ind])

init_params = {'rank': mcfg.model_rank, 
            'nz_model': nz_model,
            'scale_init': mcfg.scale_init,
            'K': K,
            'lcfg': lcfg,
            'zs_nz': zs_nz,
            'ocfg': ocfg,
            'obs': obs}

In [6]:
model = LowRankToyModel()
lrccn = res_load['track']['lrccn'][0]
obs_params, obs_type = conf.get_obs_params(ocfg)

model.initialize_latent(lrccn)
model.initialize_observations(obs_params, obs_type)

fixed_params = conf.get_fixed_params(mcfg.eigvals_flag, mcfg.eigvecs_flag, init_params)
# fit_params = {'num_em_iters': mcfg.num_em_iters, 
fit_params = {'num_em_iters': 1, 
            'num_newton_iters': mcfg.num_newton_iters,
            'm_step_option': mcfg.m_step_option,
            'fixed_params': fixed_params}
model.fit_em(obs, fit_params)

EM Iter 1
Running batch 0: trials 1 - 28
Running batch 1: trials 29 - 50
eigvec update:
[[[ 0.65-0.j  ]
  [-0.12-0.38j]
  [-0.52+0.38j]]]
